# Generating randomness in C#

## The unsafe `System.Random` class

The simplest way to generate randomness in C# is the `System.Random` class. We can use the static `Shared` property to obtain the shared instance and then we can call `Next` which returns pseudo-random integer.

Without parameters it returns `int` value between `0` (inclusive) and `Int32.MaxValue` (exclusive):

In [102]:
Random.Shared.Next()

1025017046

We can add additional parameters to limit the range of numbers. Here we simulate standard dice throw (1d6 for D&D folks):

In [111]:
Random.Shared.Next(1, 7)

5

There are some other methods that can return other random types. For example, `NextInt64` has a higher range, returning `long` from `0` to `Int64.MaxValue`:

In [112]:
Random.Shared.NextInt64()

4996420202689429621

We have `NextSingle` method for returning `float`:

In [114]:
Random.Shared.NextSingle()

0.21153504

...and of course `NextDouble` for returning double-precision decimal number:

In [115]:
Random.Shared.NextDouble()

0.6441673121517946

Last method is `NextBytes` which can fill `byte[]` or `Span<byte>` with pseudo-random numbers:

In [122]:
var buffer = new byte[10];
Random.Shared.NextBytes(buffer);
buffer

[ 68, 215, 174, 67, 230, 122, 209, 227, 62, 253 ]

The sole advantage of this class is that it can return _predictable_ sequence of seemingly random numbers, when given a fixed seed. Let's use the shared instance to generate 10 numbers. Every time you run this cell you'll get different numbers:

In [129]:
for(var i = 0; i < 10; i++) {
    Console.WriteLine(Random.Shared.Next());
}

1180771481
1005496673
1153140832
1181637537
986107623
1807010136
14750516
1290011508
583132343
1488869415


Instead of using the shared instance, we can create our own and pass it a fixed seed in constructor - here, `42`. It will return the same set of numbers every time:

In [135]:
var rng = new Random(42);
for(var i = 0; i < 10; i++) {
    Console.WriteLine(rng.Next());
}

1434747710
302596119
269548474
1122627734
361709742
563913476
1555655117
1101493307
372913049
1634773126


Although the results seem random to naked eye, they aren't random enough - they do not have enough entrophy. They are maybe good enough for displaying random tip of the day, but that's about it. You shouldn't definitely use the `Random` class for generating cryptographic keys, passwords, nonces etc.

## The safe `RandomNumberGenerator` class

There is a way better option. Use the `RandomNumberGenerator` class. It's what we call _cryptographically secure pseudo-random number generator_ and without going into too much detail, it's usually the closest thing we have to true randomness (unless you have access to some quite uncommon hardware). It lies in `System.Security.Cryptography` namespace, so we'll import it first:

In [136]:
using System.Security.Cryptography;

It has a few static methods being similar to the methods of shared `Random`. Here we have `GetInt32`. It's similar to `Next` method above in that it returns `int` from specified range. Here ve return value from `0` (inclusive) to `10` (exclusive):

In [144]:
RandomNumberGenerator.GetInt32(10)

4

For good measure, here is a cryptographically secure pseudo-random 1d6 throw:

In [150]:
RandomNumberGenerator.GetInt32(1, 7)

2

The `Fill` method is secure equivalent of `NextBytes` and will therefore fill `byte[]` or `Span<byte>` with pseudo-random numbers:

In [156]:
var buffer = new byte[10];
RandomNumberGenerator.Fill(buffer);
buffer

[ 51, 237, 129, 231, 63, 221, 65, 245, 241, 98 ]

There is a handy `GetBytes` method, which does not fill existing buffer, but returns a new `byte[]` of desired length:

In [160]:
RandomNumberGenerator.GetBytes(10)

[ 81, 47, 99, 246, 107, 98, 99, 54, 93, 234 ]

### Generating secure passwords

As a bonus, here is secure random password generation function. It selects random characters from pre-determined alphabet:

In [162]:
// Here are allowed password characters (confusable letters and numbers like 1/I, 0/O, Z/Y)
const string PasswordChars = "ABCDEFGHJKLMNPQRSTUVWXabcdefghijklmnopqrstuvwx23456789";

static string GenerateRandomPassword(int length) {
    var sb = new StringBuilder();
    for(var i = 0; i < length; i++) {
        sb.Append(PasswordChars[RandomNumberGenerator.GetInt32(PasswordChars.Length)]);
    }
    return sb.ToString();
}

// Generate some 16-character passwords
for(var i = 0; i < 10; i++) {
    Console.WriteLine(GenerateRandomPassword(16));
}

7Ssf5aDTBw8oLhsJ
ShxAxktk4wXmQu2S
sJ87XK49M3TiehKr
QwqRhfAGishNl7hm
nAqhCWakgApfFamD
QNrwKbSgbbKnL9VU
nj9PGqJRtJAouJ9E
NT4HDb7TBlVvGFb2
RB4lrDitFjbFguX9
5Nc5DbWNrVrETnaD
